Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import importlib
import datetime as dt
from datetime import datetime
from pandas import concat
from math import sqrt
from sklearn import preprocessing
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, make_scorer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import check_array
# from epftoolbox.models import LEAR
from epftoolbox.models import evaluate_lear_in_test_dataset
from epftoolbox.evaluation import sMAPE
import numpy as np
import pandas as pd
from statsmodels.robust import mad
import os

from sklearn.linear_model import LassoLarsIC, Lasso
from epftoolbox.data import scaling
from epftoolbox.data import read_data
from epftoolbox.evaluation import MAE, sMAPE

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning]

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.layers import concatenate, Flatten, Dense, BatchNormalization, Dropout
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, GRU, Dense, LSTM, concatenate
from tensorflow.keras import optimizers, initializers
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import *
import tensorflow.keras.backend as K

Load Dataset

In [ ]:
date_format = "%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True, date_parser=date_parse)

dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()

Random Forest Hyper-parameter Search

In [ ]:
#months1-7(210)
Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:8738,:]
Y_train=Y.iloc[:8738,:]
X_test=X.iloc[8738:8739,:]
Y_test=Y.iloc[8738:8739,:]

#months1-10(300)
# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:13154,:]
# Y_train=Y.iloc[:13154,:]
# X_test=X.iloc[13154:13155,:]
# Y_test=Y.iloc[13154:13155,:]

#months1-13(390)
# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:17522,:]
# Y_train=Y.iloc[:17522,:]
# X_test=X.iloc[17522:17523,:]
# Y_test=Y.iloc[17522:17523,:]

#months1-16(450)
# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:21842,:]
# Y_train=Y.iloc[:21842,:]
# X_test=X.iloc[21842:21843,:]
# Y_test=Y.iloc[21842:21843,:]

def hyperParameterTuning(X_train, Y_train):
       param_tuning = {'bootstrap': [True],
                   'max_depth': [40, 60, 80],
                   'max_features': [40, 60, 80],
                   'min_samples_leaf' : [20, 40, 60],
                   'min_samples_split' : [2, 4, 6],
                   'n_estimators': [800, 1600, 2400]
                  }
 
       RF_Regressor = RandomForestRegressor()

       gsearch = GridSearchCV(estimator = RF_Regressor,
                          param_grid = param_tuning,                        
                           scoring = 'neg_mean_absolute_error', #MAE
                          cv = 10,
                          n_jobs = -1,
                          verbose = 3)

       gsearch.fit(X_train,Y_train)
       return gsearch.best_params_

hyperParameterTuning(X_train, Y_train)

XGB Hyper-parameter Search

In [ ]:
#months1-7(210)
Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:8738,:]
Y_train=Y.iloc[:8738,:]
X_test=X.iloc[8738:8739,:]
Y_test=Y.iloc[8738:8739,:]

#months1-10(300)
# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:13154,:]
# Y_train=Y.iloc[:13154,:]
# X_test=X.iloc[13154:13155,:]
# Y_test=Y.iloc[13154:13155,:]

#months1-13(390)
# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:17522,:]
# Y_train=Y.iloc[:17522,:]
# X_test=X.iloc[17522:17523,:]
# Y_test=Y.iloc[17522:17523,:]

#months1-16(450)
# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:21842,:]
# Y_train=Y.iloc[:21842,:]
# X_test=X.iloc[21842:21843,:]
# Y_test=Y.iloc[21842:21843,:]

def hyperParameterTuning(X_train, Y_train):
       param_tuning = {'max_depth': [1, 2, 4],
                       'learning_rate': [.01, 0.03, 0.05, 0.08, 0.1],
                       'n_estimators': [25, 50, 75, 100, 200]
                  }
 
       RF_Regressor = XGBRegressor()
    
       gsearch = GridSearchCV(estimator = RF_Regressor,
                          param_grid = param_tuning,                        
                           scoring = 'neg_mean_absolute_error', #MAE
#                            scoring = 'neg_mean_squared_error',  #MSE
                          cv = 10,
                          n_jobs = -1,
                          verbose = 0)

       gsearch.fit(X_train,Y_train)
       return gsearch.best_params_

hyperParameterTuning(X_train, Y_train)

Support Vector Regression Models Hyper-parameter Search

In [ ]:
#months1-7(210)
Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:8738,:]
Y_train=Y.iloc[:8738,:]
X_test=X.iloc[8738:8739,:]
Y_test=Y.iloc[8738:8739,:]

#months1-10(300)
# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:13154,:]
# Y_train=Y.iloc[:13154,:]
# X_test=X.iloc[13154:13155,:]
# Y_test=Y.iloc[13154:13155,:]

#months1-13(390)
# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:17522,:]
# Y_train=Y.iloc[:17522,:]
# X_test=X.iloc[17522:17523,:]
# Y_test=Y.iloc[17522:17523,:]

#months1-16(450)
# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:21842,:]
# Y_train=Y.iloc[:21842,:]
# X_test=X.iloc[21842:21843,:]
# Y_test=Y.iloc[21842:21843,:]

def hyperParameterTuning(X_train, Y_train):
       param_tuning = { 'C': [0.01, 0.1],
                        'gamma': [0.0001, 0.001, 0.005],
                        'epsilon': [0.001, 0.1, 0.3],
                        'kernel': ["rbf"]
                      }
 
       RF_Regressor = SVR()

       gsearch = GridSearchCV(estimator = RF_Regressor,
                          param_grid = param_tuning,                        
                           scoring = 'neg_mean_absolute_error', #MAE
#                            scoring = 'neg_mean_squared_error',  #MSE
                          cv = 2,
                          n_jobs = -1,
                          verbose = 0)

       gsearch.fit(X_train,Y_train)
       return gsearch.best_params_

hyperParameterTuning(X_train, Y_train)

Single Headed DNN Models Hyper-paramater Search

In [ ]:
Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:7250,:]
Y_train=Y.iloc[:7250,:]
X_test=X.iloc[7250:8739,:]
Y_test=Y.iloc[7250:8739,:]

# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:11666,:]
# Y_train=Y.iloc[:11666,:]
# X_test=X.iloc[11666:13155,:]
# Y_test=Y.iloc[11666:13155,:]

# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:16082,:]
# Y_train=Y.iloc[:16082,:]
# X_test=X.iloc[16082:17523,:]
# Y_test=Y.iloc[16082:17523,:]

# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:20498,:]
# Y_train=Y.iloc[:20498,:]
# X_test=X.iloc[20498:21843,:]
# Y_test=Y.iloc[20498:21843,:]

rnn_train1_a=X_train.loc[:,"lag_-3x1":"lag_-18x1"]
rnn_train1_b=X_train.loc[:,"lag_-19x1":"lag_-34x1"]
rnn_train1_c=X_train.loc[:,"lag_-35x1":"lag_-50x1"]        
rnn_train2_a=X_train.loc[:,"lag_-3x2":"lag_-18x2"]
rnn_train2_b=X_train.loc[:,"lag_-19x2":"lag_-34x2"]
rnn_train2_c=X_train.loc[:,"lag_-35x2":"lag_-50x2"]        
rnn_train3_a=X_train.loc[:,"lag_-2x3":"lag_-17x3"]
rnn_train3_b=X_train.loc[:,"lag_-18x3":"lag_-33x3"]
rnn_train3_c=X_train.loc[:,"lag_-34x3":"lag_-49x3"]      
rnn_train4_a=X_train.loc[:,"lag_0x6":"lag_-15x6"]
rnn_train4_b=X_train.loc[:,"lag_-16x6":"lag_-31x6"]
rnn_train4_c=X_train.loc[:,"lag_-32x6":"lag_-47x6"]        
rnn_train5_a=X_train.loc[:,"lag_-2x12":"lag_-17x12"]
rnn_train5_b=X_train.loc[:,"lag_-18x12":"lag_-33x12"]
rnn_train5_c=X_train.loc[:,"lag_-34x12":"lag_-49x12"]        
rnn_train6=X_train.loc[:,"lag_2x7":"lag_17x7"]
rnn_train7=X_train.loc[:,"lag_2x8":"lag_17x8"]
rnn_train8=X_train.loc[:,"lag_2x9":"lag_17x9"]
rnn_train9=X_train.loc[:,"lag_2x10":"lag_17x10"]
rnn_train10=X_train.loc[:,"lag_2x11":"lag_17x11"]        
        
rnn_test1_a= X_test.loc[:,"lag_-3x1":"lag_-18x1"]
rnn_test1_b= X_test.loc[:,"lag_-19x1":"lag_-34x1"]
rnn_test1_c= X_test.loc[:,"lag_-35x1":"lag_-50x1"]
rnn_test2_a= X_test.loc[:,"lag_-3x2":"lag_-18x2"]
rnn_test2_b= X_test.loc[:,"lag_-19x2":"lag_-34x2"]
rnn_test2_c= X_test.loc[:,"lag_-35x2":"lag_-50x2"]
rnn_test3_a= X_test.loc[:,"lag_-2x3":"lag_-17x3"]
rnn_test3_b= X_test.loc[:,"lag_-18x3":"lag_-33x3"]
rnn_test3_c= X_test.loc[:,"lag_-34x3":"lag_-49x3"]
rnn_test4_a= X_test.loc[:,"lag_0x6":"lag_-15x6"]
rnn_test4_b= X_test.loc[:,"lag_-16x6":"lag_-31x6"]
rnn_test4_c= X_test.loc[:,"lag_-32x6":"lag_-47x6"]
rnn_test5_a= X_test.loc[:,"lag_-2x12":"lag_-17x12"]
rnn_test5_b= X_test.loc[:,"lag_-18x12":"lag_-33x12"]
rnn_test5_c= X_test.loc[:,"lag_-34x12":"lag_-49x12"]
rnn_test6=X_test.loc[:,"lag_2x7":"lag_17x7"]
rnn_test7=X_test.loc[:,"lag_2x8":"lag_17x8"]
rnn_test8=X_test.loc[:,"lag_2x9":"lag_17x9"]
rnn_test9=X_test.loc[:,"lag_2x10":"lag_17x10"]
rnn_test10=X_test.loc[:,"lag_2x11":"lag_17x11"]

rnn_Y=Y_train.loc[:,"lag_2y" : "lag_17y"]

X_scaler1_a = preprocessing.MinMaxScaler()
X_scaler1_b = preprocessing.MinMaxScaler()
X_scaler1_c = preprocessing.MinMaxScaler()

X_scaler2_a = preprocessing.MinMaxScaler()
X_scaler2_b = preprocessing.MinMaxScaler()
X_scaler2_c = preprocessing.MinMaxScaler()

X_scaler3_a = preprocessing.MinMaxScaler()
X_scaler3_b = preprocessing.MinMaxScaler()
X_scaler3_c = preprocessing.MinMaxScaler()

X_scaler4_a = preprocessing.MinMaxScaler()
X_scaler4_b = preprocessing.MinMaxScaler()
X_scaler4_c = preprocessing.MinMaxScaler()

X_scaler5_a = preprocessing.MinMaxScaler()
X_scaler5_b = preprocessing.MinMaxScaler()
X_scaler5_c = preprocessing.MinMaxScaler()


X_scaler6 = preprocessing.MinMaxScaler()
X_scaler7 = preprocessing.MinMaxScaler()
X_scaler8 = preprocessing.MinMaxScaler()
X_scaler9 = preprocessing.MinMaxScaler()
X_scaler10 = preprocessing.MinMaxScaler()

Y_scaler = preprocessing.MinMaxScaler()

rnn_scaled_train1_a = X_scaler1_a.fit_transform(rnn_train1_a)
rnn_scaled_train1_b = X_scaler1_b.fit_transform(rnn_train1_b)
rnn_scaled_train1_c = X_scaler1_c.fit_transform(rnn_train1_c)

rnn_scaled_train2_a = X_scaler2_a.fit_transform(rnn_train2_a)
rnn_scaled_train2_b = X_scaler2_b.fit_transform(rnn_train2_b)
rnn_scaled_train2_c = X_scaler2_c.fit_transform(rnn_train2_c)

rnn_scaled_train3_a = X_scaler3_a.fit_transform(rnn_train3_a)
rnn_scaled_train3_b = X_scaler3_b.fit_transform(rnn_train3_b)
rnn_scaled_train3_c = X_scaler3_c.fit_transform(rnn_train3_c)

rnn_scaled_train4_a = X_scaler4_a.fit_transform(rnn_train4_a)
rnn_scaled_train4_b = X_scaler4_b.fit_transform(rnn_train4_b)
rnn_scaled_train4_c = X_scaler4_c.fit_transform(rnn_train4_c)

rnn_scaled_train5_a = X_scaler5_a.fit_transform(rnn_train5_a)
rnn_scaled_train5_b = X_scaler5_b.fit_transform(rnn_train5_b)
rnn_scaled_train5_c = X_scaler5_c.fit_transform(rnn_train5_c)

rnn_scaled_train6 = X_scaler6.fit_transform(rnn_train6)
rnn_scaled_train7 = X_scaler7.fit_transform(rnn_train7)
rnn_scaled_train8 = X_scaler8.fit_transform(rnn_train8)
rnn_scaled_train9 = X_scaler9.fit_transform(rnn_train9)
rnn_scaled_train10 = X_scaler10.fit_transform(rnn_train10)

Y_train_Scaled   = Y_scaler.fit_transform(Y_train)
Y_test_scaled = Y_scaler.transform(Y_test)

X_train_Scaled = np.hstack(
    (rnn_scaled_train1_a, rnn_scaled_train1_b, rnn_scaled_train1_c, rnn_scaled_train2_a, rnn_scaled_train2_b, rnn_scaled_train2_c,
     rnn_scaled_train3_a, rnn_scaled_train3_b, rnn_scaled_train3_c, rnn_scaled_train4_a, rnn_scaled_train4_b, rnn_scaled_train4_c,
     rnn_scaled_train5_a, rnn_scaled_train5_b, rnn_scaled_train5_c,rnn_scaled_train6,rnn_scaled_train7, rnn_scaled_train8,
     rnn_scaled_train9, rnn_scaled_train10)
).reshape(rnn_train6.shape[0], 20, 16).transpose(0, 2, 1)

X_test_Scaled = np.hstack(
    (X_scaler1_a.transform(rnn_test1_a),X_scaler1_b.transform(rnn_test1_b),X_scaler1_c.transform(rnn_test1_c),
     X_scaler2_a.transform(rnn_test2_a),X_scaler2_b.transform(rnn_test2_b),X_scaler2_c.transform(rnn_test2_c),
     X_scaler3_a.transform(rnn_test3_a),X_scaler3_b.transform(rnn_test3_b),X_scaler3_c.transform(rnn_test3_c),
     X_scaler4_a.transform(rnn_test4_a),X_scaler4_b.transform(rnn_test4_b),X_scaler4_c.transform(rnn_test4_c),
     X_scaler5_a.transform(rnn_test5_a),X_scaler5_b.transform(rnn_test5_b),X_scaler5_c.transform(rnn_test5_c),
     X_scaler6.transform(rnn_test6),X_scaler7.transform(rnn_test7),X_scaler8.transform(rnn_test8),
     X_scaler9.transform(rnn_test9), X_scaler10.transform(rnn_test10))
).reshape(rnn_test6.shape[0], 20, 16).transpose(0, 2, 1)


score_acc = make_scorer(mean_absolute_error)
mse = make_scorer(MSE, greater_is_better=False)
i_shape=(X_train_Scaled.shape[1], X_train_Scaled.shape[2])
      
pp ={
    'neurons_0': list(range(16, 256, 16)),
    'activation_0':['relu', 'sigmoid',  'tanh', LeakyReLU],
    'neurons_1': list(range(16, 256, 16)),
    'activation_1':['relu', 'sigmoid',  'tanh', LeakyReLU],
    'neurons_2': list(range(16, 256, 16)),
    'activation_2':['relu', 'sigmoid',  'tanh', LeakyReLU],
    'neurons_3': list(range(16, 256, 16)),
    'activation_3':['relu', 'sigmoid',  'tanh', LeakyReLU],
    'neurons_4': list(range(16, 256, 16)),
    'activation_4':['relu', 'sigmoid',  'tanh', LeakyReLU],

    'learning_rate': list(np.linspace(0.0001,0.02, 10)),        
    'layers1': list(range(1,3, 1)),
    'layers2': list(range(1,3, 1)),
    'layers3': list(range(1,3, 1)),
    'layers4': list(range(1,3, 1)),
    'dropout_rate': list(np.linspace(0.0,0.2, 10)),
    'batch_size': list(range(16, 64, 16)),
    'epochs': [200]
}

def create_model(x_train, y_train, x_val, y_val, params):
            nn = Sequential()
            nn.add(Flatten(input_shape=i_shape))

            for i in range(params['layers1']):
                nn.add(Dense(params['neurons_0'], input_shape=i_shape, activation=params['activation_0']))
                
            for i in range(params['layers2']):
                nn.add(Dense(params['neurons_1'], activation=params['activation_1']))
   
            for i in range(params['layers3']):
                nn.add(Dense(params['neurons_2'], activation=params['activation_2']))
            
            nn.add(Dropout(params['dropout_rate'], seed=123))
            
            for i in range(params['layers4']):
                nn.add(Dense(params['neurons_3'], activation=params['activation_3']))
                
            nn.add(Dense(params['neurons_4'], activation=params['activation_4']))
            nn.add(Dense(16))
            opt = Adam(lr = params['learning_rate'])
            nn.compile(loss='mean_absolute_error', optimizer=opt, metrics=['mean_absolute_error'])
            out=nn.fit(x_train, y_train,validation_data=[x_val, y_val], batch_size=params['batch_size'], epochs=params['epochs'], verbose=2, 
                          callbacks=[talos.utils.early_stopper(epochs=params['epochs'], mode='moderate', 
                                                               min_delta=0.001, monitor='mean_absolute_error')])
            return out, nn
        
h = talos.Scan(x=X_train_Scaled,y=Y_train_Scaled,x_val=X_test_Scaled,  y_val=Y_test_scaled, params=pp, model=create_model,val_split=0.2,
            experiment_name ='bm_1-3', random_method='quantum', round_limit=30, print_params=True)
h.data.sort_values(by='val_mean_absolute_error', ascending=True)

Multi-headed DNN RNN Hyper-paramater Search

In [ ]:
Y=dat.iloc[:, 0:16]
X=dat.iloc[:,16:]
X_train=X.iloc[:7250,:]
Y_train=Y.iloc[:7250,:]
X_test=X.iloc[7250:8739,:]
Y_test=Y.iloc[7250:8739,:]

# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:11666,:]
# Y_train=Y.iloc[:11666,:]
# X_test=X.iloc[11666:13155,:]
# Y_test=Y.iloc[11666:13155,:]

# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:16082,:]
# Y_train=Y.iloc[:16082,:]
# X_test=X.iloc[16082:17523,:]
# Y_test=Y.iloc[16082:17523,:]

# Y=dat.iloc[:, 0:16]
# X=dat.iloc[:,16:]
# X_train=X.iloc[:20498,:]
# Y_train=Y.iloc[:20498,:]
# X_test=X.iloc[20498:21843,:]
# Y_test=Y.iloc[20498:21843,:]

# Extracting specific lagged features for LSTM and FFNN models
rnn_train_LSTM_1 = X_train.loc[:,"lag_-3x1":"lag_-50x1"]
rnn_train_LSTM_2 = X_train.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_train_LSTM_3 = X_train.loc[:,"lag_-2x3":"lag_-49x3"]
rnn_train_LSTM_4 = X_train.loc[:,"lag_0x6":"lag_-47x6"]
rnn_train_LSTM_5 = X_train.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_test_LSTM_1 = X_test.loc[:,"lag_-3x1": "lag_-50x1"]
rnn_test_LSTM_2 = X_test.loc[:,"lag_-3x2": "lag_-50x2"]
rnn_test_LSTM_3 = X_test.loc[:,"lag_-2x3": "lag_-49x3"]
rnn_test_LSTM_4 = X_test.loc[:,"lag_0x6" : "lag_-47x6"]
rnn_test_LSTM_5 = X_test.loc[:,"lag_-2x12": "lag_-49x12"]

rnn_train_ffnn_1 = X_train.loc[:,"lag_2x7": "lag_17x7"]
rnn_train_ffnn_2 = X_train.loc[:,"lag_2x8": "lag_17x8"]
rnn_train_ffnn_3 = X_train.loc[:,"lag_2x9": "lag_17x9"]
rnn_train_ffnn_4 = X_train.loc[:,"lag_2x10": "lag_17x10"]
rnn_train_ffnn_5 = X_train.loc[:,"lag_2x11": "lag_17x11"]

rnn_test_ffnn_1 = X_test.loc[:,"lag_2x7":"lag_17x7"]
rnn_test_ffnn_2 = X_test.loc[:,"lag_2x8": "lag_17x8"]
rnn_test_ffnn_3 = X_test.loc[:,"lag_2x9": "lag_17x9"]
rnn_test_ffnn_4 = X_test.loc[:,"lag_2x10":"lag_17x10"]
rnn_test_ffnn_5 = X_test.loc[:,"lag_2x11": "lag_17x11"]

rnn_Y = Y_train.loc[:,"lag_2y": "lag_17y"]

X_scaler_LSTM_1 = preprocessing.MinMaxScaler()
X_scaler_LSTM_2 = preprocessing.MinMaxScaler()
X_scaler_LSTM_3 = preprocessing.MinMaxScaler()
X_scaler_LSTM_4 = preprocessing.MinMaxScaler()
X_scaler_LSTM_5 = preprocessing.MinMaxScaler()

X_scaler_ffnn_1 = preprocessing.MinMaxScaler()
X_scaler_ffnn_2 = preprocessing.MinMaxScaler()
X_scaler_ffnn_3 = preprocessing.MinMaxScaler()
X_scaler_ffnn_4 = preprocessing.MinMaxScaler()
X_scaler_ffnn_5 = preprocessing.MinMaxScaler()

Y_scaler = preprocessing.MinMaxScaler()

rnn_scaled_train_LSTM_1 = X_scaler_LSTM_1.fit_transform(rnn_train_LSTM_1)
rnn_scaled_train_LSTM_2 = X_scaler_LSTM_2.fit_transform(rnn_train_LSTM_2)
rnn_scaled_train_LSTM_3 = X_scaler_LSTM_3.fit_transform(rnn_train_LSTM_3)
rnn_scaled_train_LSTM_4 = X_scaler_LSTM_4.fit_transform(rnn_train_LSTM_4)
rnn_scaled_train_LSTM_5 = X_scaler_LSTM_5.fit_transform(rnn_train_LSTM_5)

rnn_scaled_train_ffnn_1 = X_scaler_ffnn_1.fit_transform(rnn_train_ffnn_1)
rnn_scaled_train_ffnn_2 = X_scaler_ffnn_2.fit_transform(rnn_train_ffnn_2)
rnn_scaled_train_ffnn_3 = X_scaler_ffnn_3.fit_transform(rnn_train_ffnn_3)
rnn_scaled_train_ffnn_4 = X_scaler_ffnn_4.fit_transform(rnn_train_ffnn_4)
rnn_scaled_train_ffnn_5 = X_scaler_ffnn_5.fit_transform(rnn_train_ffnn_5)

Y_train_Scaled = Y_scaler.fit_transform(Y_train)
Y_test_scaled = Y_scaler.transform(Y_test)

X_train_Scaled_LSTM = np.hstack(
    (rnn_scaled_train_LSTM_1, rnn_scaled_train_LSTM_2, rnn_scaled_train_LSTM_3,
     rnn_scaled_train_LSTM_4, rnn_scaled_train_LSTM_5)
).reshape(rnn_train_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_train_Scaled_ffnn = np.hstack(
    (rnn_scaled_train_ffnn_1, rnn_scaled_train_ffnn_2, rnn_scaled_train_ffnn_3,
     rnn_scaled_train_ffnn_4, rnn_scaled_train_ffnn_5)
).reshape(rnn_train_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

X_test_Scaled_LSTM = np.hstack(
    (X_scaler_LSTM_1.transform(rnn_test_LSTM_1), X_scaler_LSTM_2.transform(rnn_test_LSTM_2),
     X_scaler_LSTM_3.transform(rnn_test_LSTM_3), X_scaler_LSTM_4.transform(rnn_test_LSTM_4),
     X_scaler_LSTM_5.transform(rnn_test_LSTM_5))
).reshape(rnn_test_LSTM_1.shape[0], 5, 48).transpose(0, 2, 1)

X_test_Scaled_ffnn = np.hstack(
    (X_scaler_ffnn_1.transform(rnn_test_ffnn_1), X_scaler_ffnn_2.transform(rnn_test_ffnn_2),
     X_scaler_ffnn_3.transform(rnn_test_ffnn_3), X_scaler_ffnn_4.transform(rnn_test_ffnn_4),
     X_scaler_ffnn_5.transform(rnn_test_ffnn_5))
).reshape(rnn_test_ffnn_1.shape[0], 5, 16).transpose(0, 2, 1)

pp = {'lstm_neurons_0': [16, 32, 64, 128, 192, 256],

      'ffnn_neurons_0': [16, 32, 64, 128, 192, 256],

      'dense_f_neurons': [16, 32, 64, 128, 192, 256],
      'lstm_activation_0': ['relu', 'sigmoid', 'tanh'],

      'activation_0': ['relu', 'sigmoid', 'tanh', LeakyReLU],
      'activation_1': ['relu', 'sigmoid', 'tanh', LeakyReLU],

      'learning_rate': list(np.linspace(0.0001, 0.02, 10)),
      'dropout_rate_lstm': list(np.linspace(0.0,0.2, 10)),
      'dropout_rate_ffnn': list(np.linspace(0.0,0.2, 10)),
      
      'batch_size': [4, 8, 16, 32, 48],
      'epochs': [300]
      }
i_shape_lstm = (X_train_Scaled_LSTM.shape[1], X_train_Scaled_LSTM.shape[2])
i_shape_ffnn = (X_train_Scaled_ffnn.shape[1], X_train_Scaled_ffnn.shape[2])


def create_model(x_train, y_train, x_val, y_val, params):
    visible1 = Input(shape=(i_shape_lstm))
    dense1 = LSTM(params['lstm_neurons_0'], return_sequences=True, activation=params['lstm_activation_0'], input_shape=i_shape_lstm)(visible1)
    dense2 = LSTM(params['lstm_neurons_0'], return_sequences=True, activation=params['lstm_activation_0'], input_shape=i_shape_lstm)(dense1)
    do_lstm = Dropout(params['dropout_rate_lstm'])(dense2)
    dense3 = LSTM(params['lstm_neurons_0'], return_sequences=True, activation=params['lstm_activation_0'], input_shape=i_shape_lstm)(do_lstm)
    flat1 = Flatten()(dense3)

    visible2 = Input(shape=(i_shape_ffnn))
    dense5 = Dense(params['ffnn_neurons_0'], activation=params['activation_0'])(visible2)
    dense6 = Dense(params['ffnn_neurons_0'], activation=params['activation_0'])(dense5)
    do_ffnn = Dropout(params['dropout_rate_ffnn'])(dense6)
    dense7 = Dense(params['ffnn_neurons_0'], activation=params['activation_0'])(do_ffnn)
    flat2 = Flatten()(dense7)

    merged = concatenate([flat1, flat2])
    dense_f = Dense(params['dense_f_neurons'], activation=params['activation_1'])(merged)
    output = Dense(16)(dense_f)
    model = Model(inputs=[visible1, visible2], outputs=output)

    opt = Adam(lr=params['learning_rate'])
    model.compile(loss='mean_absolute_error', optimizer=opt, metrics=['mean_absolute_error'])


    out = model.fit(x=x_train, y=y_train, validation_data=[x_val, y_val],
                    batch_size=params['batch_size'], epochs=params['epochs'], verbose=2,
                    callbacks=[talos.utils.early_stopper(epochs=params['epochs'], mode='moderate',
                                                         min_delta=0.001, monitor='mean_absolute_error')])
    return out, model


h = talos.Scan(x=[X_train_Scaled_LSTM, X_train_Scaled_ffnn], y=Y_train_Scaled, x_val=[X_train_Scaled_LSTM, X_train_Scaled_ffnn], y_val=Y_train_Scaled, params=pp, model=create_model,
               experiment_name='MH_DNN_1-3', round_limit=30, print_params=True)
h.data.sort_values(by='val_mean_absolute_error', ascending=True)
